In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [2]:
device = torch.device('cuda')
print('{} device is available'.format(device))

cuda device is available


In [3]:
    class MnistDataset(Dataset):
        def __init__(self, csv_file, transform=None, target_transform=None):
            self.digits = pd.read_csv(csv_file)
            self.target_transform = target_transform
            
        def __len__(self):
            return len(self.digits)
    
        def __getitem__(self, idx):
            if torch.is_tensor(idx):
                idx = idx.tolist()
            label = torch.tensor(self.digits.iloc[idx]['label']).to(device)
            image = torch.tensor(self.digits.iloc[idx].drop('label').values, dtype=torch.float64).to(device)
            
            
    
            if self.target_transform:
                label = self.target_transform(label)
            
            return image, label

In [4]:
from torchvision.transforms import Lambda
batch_size =256

target_transform = Lambda(lambda y: torch.zeros(10, dtype=torch.float64).to(device).scatter_(0, y.clone().detach(), value=1))

train_dataloader = DataLoader(
    MnistDataset('./digit-recognizer/train.csv', target_transform=target_transform), 
    batch_size=batch_size
)

In [5]:
from torch import nn

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 1024, dtype=torch.float64), 
            nn.ReLU(),
            nn.Linear(1024, 1024, dtype=torch.float64),
            nn.ReLU(),
            nn.Linear(1024, 10, dtype=torch.float64),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)

In [8]:
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [9]:
def test_loop(testfile, model):
    model.eval()

    data = pd.read_csv(testfile)
    data = torch.tensor(data.values, dtype=torch.float64).to(device)
    size = len(data)
    
    pred = torch.argmax(model(data), dim=1)
    pred = pred.cpu().numpy() 
    results = pd.DataFrame(columns = ['Label'], data=pred)
    results.index = results.index + 1
    print('the index is converted')
    submission_file = "submission.csv"
    results.to_csv(submission_file, index_label='ImageID')
    

    print(f"Your result is saved at the {submission_file}")

In [10]:
def train_loop(dataloader, model):
    model.train()
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
epochs = 10
learning_rate = 1e-3

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model)

print("Done!")

Epoch 1
-------------------------------


/home/hanna/miniconda3/lib/python3.12/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at ../aten/src/ATen/Context.cpp:296.)
  return F.linear(input, self.weight, self.bias)


loss: 15.429134  [  256/42000]
loss: 0.181862  [25856/42000]
Epoch 2
-------------------------------
loss: 0.165186  [  256/42000]
loss: 0.097523  [25856/42000]
Epoch 3
-------------------------------
loss: 0.055355  [  256/42000]
loss: 0.057772  [25856/42000]
Epoch 4
-------------------------------
loss: 0.075075  [  256/42000]
loss: 0.048451  [25856/42000]
Epoch 5
-------------------------------
loss: 0.034809  [  256/42000]
loss: 0.052990  [25856/42000]
Epoch 6
-------------------------------
loss: 0.029962  [  256/42000]
loss: 0.017753  [25856/42000]
Epoch 7
-------------------------------
loss: 0.034573  [  256/42000]
loss: 0.028579  [25856/42000]
Epoch 8
-------------------------------
loss: 0.073928  [  256/42000]
loss: 0.057237  [25856/42000]
Epoch 9
-------------------------------
loss: 0.078055  [  256/42000]
loss: 0.041968  [25856/42000]
Epoch 10
-------------------------------
loss: 0.013203  [  256/42000]
loss: 0.042546  [25856/42000]
Done!


In [13]:
test_loop('./digit-recognizer/test.csv', model)

the index is converted
Your result is saved at the submission.csv
